In [48]:
import numpy as np
import pandas as pd
import os
import nltk

In [49]:
# needed for nltk.word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mfekadu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [50]:
DATA_DIR = "state-of-the-state"
FILENAME = "index.csv"
SPEECH_DIR_NAME = "speeches"
TABLE_PATH = os.path.join(DATA_DIR, FILENAME)
SPEECH_DIR = os.path.join(DATA_DIR, SPEECH_DIR_NAME)
print("DATA_DIR:\t", os.listdir(DATA_DIR))
print("SPEECH_DIR[:2]:\t", os.listdir(SPEECH_DIR)[:2])
print("TABLE_PATH:\t", TABLE_PATH)
print("SPEECH_DIR:\t", SPEECH_DIR)

DATA_DIR:	 ['words.csv', 'speeches', 'README.md', 'index.csv']
SPEECH_DIR[:2]:	 ['Minnesota_SOTS.txt', 'Indiana_SOTS.txt']
TABLE_PATH:	 state-of-the-state/index.csv
SPEECH_DIR:	 state-of-the-state/speeches


In [51]:
# read the csv for info on the data
df = pd.read_csv(TABLE_PATH)
df.head()

,state,governor,party,filename,url
0,Alabama,Kay Ivey,R,Alabama_SOTS.txt,https://governor.alabama.gov/remarks-speeches/...
1,Alaska,Mike Dunleavy,R,Alaska_SOTS.txt,https://gov.alaska.gov/newsroom/2019/01/22/201...
2,Arizona,Doug Ducey,R,Arizona_SOTS.txt,https://azgovernor.gov/governor/news/2019/01/g...
3,Arkansas,Asa Hutchinson,R,Arkansas_SOTS.txt,https://governor.arkansas.gov/news-media/speec...
4,California,Gavin Newsom,D,California_SOTS.txt,https://www.gov.ca.gov/2019/02/12/state-of-the...


In [52]:
# dictionary mapping filenames to text content
speech_map = {}

# read the files
for fname in df['filename'].get_values():
    with open( os.path.join(SPEECH_DIR, fname) ) as f:
        speech_map[fname] = f.read()
        f.close()

In [56]:
for state in speech_map:
    tokens = nltk.word_tokenize(speech_map[state])
    print(tokens[:5])
    bigram_generator = nltk.bigrams(tokens)
    bigram_list = list(bigram_generator)
    print(bigram_list[:5])
    break
        

['Lieutenant', 'Governor', 'Ainsworth', ',', 'Pro']
[('Lieutenant', 'Governor'), ('Governor', 'Ainsworth'), ('Ainsworth', ','), (',', 'Pro'), ('Pro', 'Tempore')]


In [64]:
tokens.remove(',')

In [65]:
tokens

['Lieutenant',
 'Governor',
 'Ainsworth',
 'Pro',
 'Tempore',
 'Marsh',
 'Speaker',
 'McCutcheon',
 ',',
 'Speaker',
 'Pro',
 'Tempore',
 'Gaston',
 ',',
 'members',
 'of',
 'the',
 'Alabama',
 'Legislature',
 ',',
 'Chief',
 'Justice',
 'Parker',
 ',',
 'justices',
 'of',
 'the',
 'Alabama',
 'Supreme',
 'Court',
 ',',
 'distinguished',
 'guests',
 '–',
 'and',
 'my',
 'fellow',
 'Alabamians',
 ':',
 'Just',
 'over',
 '48',
 'hours',
 'ago',
 ',',
 'Mother',
 'Nature',
 '’',
 's',
 'wrath',
 '–',
 'in',
 'the',
 'form',
 'of',
 'vicious',
 'and',
 'deadly',
 'tornados',
 '–',
 'ripped',
 'through',
 'our',
 'state',
 ',',
 'leaving',
 'behind',
 'significant',
 'devastation',
 '.',
 'At',
 'least',
 '23',
 'innocent',
 'lives',
 'were',
 'lost',
 '.',
 'Young',
 'children',
 'who',
 'had',
 'barely',
 'experienced',
 'life',
 '.',
 'Mothers',
 '.',
 'Fathers',
 '.',
 'Friends',
 'and',
 'neighbors',
 '.',
 'It',
 'is',
 'during',
 'times',
 'like',
 'these',
 'that',
 'we',
 'turn',
 